In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math

In [2]:
df = pd.read_csv(r'shedulers.csv', encoding='cp1251', sep=';')
df.columns = ['FIO','SHEDULE','DATE_BEGIN','DATE_END']
df.head(10)

,FIO,SHEDULE,DATE_BEGIN,DATE_END
0,Поставщик 1,дддвсвнн,01.01.2019 0:00,10.01.2019 0:00
1,Поставщик 1,ннвннв,11.01.2019 0:00,15.01.2019 0:00
2,Поставщик 1,св,16.01.2019 0:00,20.01.2019 0:00
3,Поставщик 2,свсвсв,01.01.2019 0:00,07.01.2019 0:00
4,Поставщик 2,днвсв,08.01.2019 0:00,14.01.2019 0:00
5,Поставщик 2,ннддвсв,15.01.2019 0:00,31.12.9999 0:00
6,Поставщик 3,нвнвнв,01.01.2019 0:00,01.02.2019 0:00
7,Поставщик 3,двдвдвдв,02.02.2019 0:00,31.12.9999 0:00


Задание 3.

In [3]:
def slicer(x):
  l = x.split()
  s = int(l[1])
  f = s + int(l[2])
  return l[0][s:f]

def COUNT_WORK_DAYS(df,start,end):
  ds = df.copy()
  ds['DATE_BEGIN'] = ds['DATE_BEGIN'].apply(lambda x: datetime.strptime(x,'%d.%m.%Y %H:%M')).astype(object)
  ds['DATE_END'] = ds['DATE_END'].apply(lambda x: datetime.strptime(x,'%d.%m.%Y %H:%M'))
    
  ds['date_start'] = np.maximum(ds['DATE_BEGIN'],start)
  ds['pos_start'] = (ds['date_start']-ds['DATE_BEGIN']).apply(lambda x: x.days)
  ds['Diff'] = (np.minimum(ds['DATE_END'],end)-ds['date_start']).apply(lambda x: x.days)+1
  ds = ds[ds.Diff>0]
  ds['n_reps'] = np.ceil(((ds['pos_start']+ds.Diff+1)/ds.SHEDULE.apply(lambda x: len(x)))).astype(int)

  ds['SHEDULE'] = ds['SHEDULE']*ds['n_reps']+' '+ds['pos_start'].astype(str)+' ' + ds['Diff'].astype(str)
  ds['SHEDULE'] = ds['SHEDULE'].apply(slicer)

  ds = ds.groupby(by=['FIO'])[['date_start','SHEDULE']].agg({'date_start':'min','SHEDULE':'sum'}).reset_index().sort_values(by='FIO')

  res = []
  date_start = ds.date_start.values
  SHEDULE = ds.SHEDULE.values
  FIO = ds.FIO.values
  for i in range(ds.shape[0]):
    shed = SHEDULE[i]
    name = FIO[i]
    first = pd.to_datetime(str(date_start[i]))
    for j in range(len(shed)):
      if shed[j]=='д': 
        res.append([name,first+timedelta(hours = 8),first+timedelta(hours = 20)])

      elif shed[j]=='н': 
        res.append([name,first+timedelta(hours = 20),first+timedelta(hours = 32)])

      elif shed[j]=='с': 
        res.append([name,first+timedelta(hours = 8),first+timedelta(hours = 32)])
      first+=timedelta(days=1)
  return pd.DataFrame(res,columns=['FIO','DATE_BEGIN','DATE_END'])

Задание 4

In [4]:
start = datetime.strptime('01.01.2019','%d.%m.%Y')
end = datetime.strptime('31.01.2019','%d.%m.%Y')
res = COUNT_WORK_DAYS(df,start,end)

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,


In [9]:
# число рабочих дней
out = res.groupby(by='FIO')[['DATE_BEGIN']].count().rename({'DATE_BEGIN':'N_WORK_DAYS'},axis=1).reset_index()
out 

,FIO,N_WORK_DAYS
0,Поставщик 1,15
1,Поставщик 2,22
2,Поставщик 3,16


In [10]:
# поставщики с # рабочих дней в январе больше 10
out[out.N_WORK_DAYS>10][['FIO']]

,FIO
0,Поставщик 1
1,Поставщик 2
2,Поставщик 3


In [7]:
# поставщики, кто работал 14,15 и 16 января
t1 = datetime.strptime('14.01.2019','%d.%m.%Y')
t2 = datetime.strptime('15.01.2019','%d.%m.%Y')
t3 = datetime.strptime('16.01.2019','%d.%m.%Y')
res['DATE_BEGIN'] = res['DATE_BEGIN'].apply(lambda x: x.replace(hour=0, minute=0))

In [8]:
out = res[(res.DATE_BEGIN==t1) | (res.DATE_BEGIN==t2)| (res.DATE_BEGIN==t3)].groupby(by='FIO')[['DATE_BEGIN']].count().rename({'DATE_BEGIN':'N_WORK_DAYS'},axis=1).reset_index()
out[out.N_WORK_DAYS==3][['FIO']]

,FIO
0,Поставщик 1
1,Поставщик 2
